# Shapelet-Based Time Series Classification

Shapelets are short, discriminative subsequences. A classifier learns a set of shapelets and
represents each series by its distance to these prototypes.

## Core idea
For a shapelet $s \in \mathbb{R}^L$ and series $x_{1:T}$, the distance feature is
$d(x, s) = \min_{t} \|x_{t:t+L-1} - s\|_2$. Small distances mean the pattern appears in the series.

In [ ]:
import numpy as np
import plotly.graph_objects as go

np.random.seed(31)
T = 100
x = 0.2 * np.random.randn(T)
shapelet = np.array([0.0, 0.6, 1.0, 0.6, 0.0, -0.4, -0.8, -0.4])
insert_at = 40
x[insert_at:insert_at + len(shapelet)] += shapelet

def distance_profile(series, s):
    L = len(s)
    dists = []
    for i in range(len(series) - L + 1):
        window = series[i:i + L]
        dists.append(np.linalg.norm(window - s))
    return np.array(dists)

profile = distance_profile(x, shapelet)
min_idx = int(np.argmin(profile))

fig = go.Figure()
fig.add_trace(go.Scatter(y=x, mode='lines', name='series'))
fig.add_trace(
    go.Scatter(
        x=np.arange(insert_at, insert_at + len(shapelet)),
        y=x[insert_at:insert_at + len(shapelet)],
        mode='lines',
        name='shapelet occurrence',
        line=dict(color='red', width=3),
    )
)
fig.update_layout(title='Series with embedded shapelet', xaxis_title='time index')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=profile, mode='lines', name='distance'))
fig.add_vline(x=min_idx, line_color='red', line_dash='dash')
fig.update_layout(title='Distance profile to the shapelet', xaxis_title='window start index')
fig.show()


## sktime inventory for shapelet-based classifiers
Shapelet-based estimators typically live in modules containing `shapelet` or `shapelet_based`.

In [ ]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    def _estimators_df():
        try:
            return all_estimators(estimator_types='classifier', as_dataframe=True)
        except TypeError:
            ests = all_estimators(estimator_types='classifier')
            return pd.DataFrame(
                [{'name': n, 'class': c.__name__, 'module': c.__module__} for n, c in ests]
            )

    df = _estimators_df()
    mask = df['module'].str.contains('shapelet', case=False, na=False)
    df_shapelet = df[mask].sort_values('name')
    df_shapelet
except Exception as exc:
    print('sktime not installed or registry unavailable:', exc)


## When to use
- Highly interpretable: shapelets act as visual motifs.
- Training can be expensive; modern variants use randomized or learned candidates.
- Best when classes differ by short local patterns.